In [29]:
import numpy as np
import random
import time
from collections import deque
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 

In [30]:

# Générer le graphe TSP incomplet
def generate_tsp_incomplete(num_sommets, max_poids_arete):
    matrice = [[None for _ in range(num_sommets)] for _ in range(num_sommets)]
    intervalles = {} 
    for i in range(num_sommets):
        intervalle_start = random.randint(0, 10)
        intervalle_end = intervalle_start + random.randint(5, 100)
        intervalles[i] = (intervalle_start, intervalle_end)
    for i in range(num_sommets):
        for j in range(i + 1, num_sommets):
            if random.choice([True, False]):
                poids = random.randint(1, max_poids_arete)
                matrice[i][j] = poids
                matrice[j][i] = poids
    for i in range(num_sommets):
        pas_connexe = True
        for j in range(num_sommets):
            if matrice[i][j] is None:
                matrice[i][j] = float('inf')
            else:
                pas_connexe = False
        if pas_connexe:
            poids = random.randint(1, max_poids_arete)
            if i != (num_sommets-1):
                matrice[i][num_sommets-1] = poids
                matrice[num_sommets-1][i] = poids
            else:
                matrice[i][num_sommets-2] = poids
                matrice[num_sommets-2][i] = poids
    return np.array(matrice), intervalles  

In [31]:
num_sommets = 5
max_poids_arete = 10

# Générer les données du problème
tsp_matrice, tsp_intervalles = generate_tsp_incomplete(num_sommets, max_poids_arete)

# Afficher la matrice pondérée
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)

# Afficher les intervalles 
print("\nIntervalles des sommets")
print(tsp_intervalles)  


Matrice pondérée:
[inf inf  1.  4.  5.]
[inf inf inf 10. inf]
[ 1. inf inf  3.  9.]
[ 4. 10.  3. inf  1.]
[ 5. inf  9.  1. inf]

Intervalles des sommets
{0: (1, 100), 1: (4, 33), 2: (7, 64), 3: (1, 24), 4: (10, 79)}


In [32]:
# Placeholder for the ACO algorithm parameters
num_sommets = 10
num_ants = 50
num_iterations = 100
alpha = 1.0
beta = 5.0
evaporation_rate = 0.5
pheromone_constant = 100.0

# Generating a random TSP instance
def generate_instance(num_sommets, max_poids_arete, max_intervalle_inf, max_intervalle_sup):
    matrice = np.full((num_sommets, num_sommets), np.inf)
    intervalles = {}

    for i in range(num_sommets):
        intervalle_start = random.randint(0, max_intervalle_inf)
        intervalle_end = intervalle_start + random.randint(max_intervalle_inf, max_intervalle_sup)
        intervalles[i] = (intervalle_start, intervalle_end)

    for i in range(num_sommets):
        for j in range(i + 1, num_sommets):
            if random.random() > 0.5:  # Adding some randomness to edge creation
                poids = random.randint(1, max_poids_arete)
                matrice[i][j] = poids
                matrice[j][i] = poids

    return matrice, intervalles

# Define the ACO algorithm functions

def is_edge_valid(from_node, to_node, current_length, visited, tsp_matrice, tsp_intervalles):
    weight = tsp_matrice[from_node][to_node]
    if weight == np.inf:
        return (False, 0)
    if to_node in visited:
        min_weight, max_weight = tsp_intervalles[to_node]
        if current_length + weight > max_weight:
            return (False, 0)
        if current_length + weight < min_weight:
            return (True,  min_weight - current_length)
    return (True, weight)

def update_pheromones(best_path, best_path_length, pheromone_tsp_matrice):
    pheromone_tsp_matrice *= (1 - evaporation_rate)
    for i in range(len(best_path) - 1):
        from_node = best_path[i]
        to_node = best_path[i + 1]
        pheromone_tsp_matrice[from_node][to_node] += pheromone_constant / best_path_length 

def select_next_node(current_node, visited, current_length, pheromone_tsp_matrice, tsp_matrice, tsp_intervalles):
    probabilities = []
    total_prob = 0
    pheromone_row = pheromone_tsp_matrice[current_node]
    heuristic_row = (1.0 / tsp_matrice[current_node]) ** beta

    for node in range(num_sommets):
        if node != current_node:
            valide, poids = is_edge_valid(current_node, node, current_length, visited, tsp_matrice, tsp_intervalles)
            if valide:
                pheromone_level = pheromone_row[node] ** alpha
                heuristic_level = heuristic_row[node]
                probability = pheromone_level * heuristic_level
                probabilities.append((node, probability))
                total_prob += probability
    
    if total_prob == 0:
        return (None, 0)
    
    threshold = random.uniform(0, total_prob)
    cumulative_prob = 0
    for node, probability in probabilities:
        cumulative_prob += probability
        if cumulative_prob >= threshold:
            valide, poids = is_edge_valid(current_node, node, current_length, visited, tsp_matrice, tsp_intervalles)
            return (node, poids)
    return (None, 0) 

def ant_colony_optimization(tsp_matrice, tsp_intervalles, num_ants, num_iterations):
    best_path = None
    best_path_length = float('inf') 
    path_lengths = deque()
    best_path_lengths = deque()
    execution_times = [] 
    distances = [] 
    pheromone_tsp_matrice = np.ones((num_sommets, num_sommets))

    for iteration in range(num_iterations):
        start_time = time.time() 
        all_paths = []
        for ant in range(num_ants):
            current_node = random.randint(0, num_sommets - 1)
            path = [current_node]
            visited = {current_node}
            min_weight, max_weight = tsp_intervalles[current_node]
            current_length = min_weight
            
            while len(visited) < num_sommets:
                next_node, weight_to_add = select_next_node(current_node, visited, current_length, pheromone_tsp_matrice, tsp_matrice, tsp_intervalles)
                if next_node is None:
                    break
                current_length += weight_to_add
                current_node = next_node
                path.append(current_node)
                visited.add(current_node)
            
            valid, weight = is_edge_valid(current_node, path[0], current_length, visited, tsp_matrice, tsp_intervalles)
            if len(visited) == num_sommets and valid:
                current_length += weight
                path_lengths.append(current_length)
                path.append(path[0]) 
                all_paths.append((path, current_length))
                if current_length < best_path_length:
                    best_path_length = current_length
                    best_path = path
                best_path_lengths.append(best_path_length)    
            
        if best_path:
            update_pheromones(best_path, best_path_length, pheromone_tsp_matrice)
        end_time = time.time()
        execution_times.append(end_time - start_time)
        distances.append(best_path_length) 
    
    return best_path, best_path_length, path_lengths, best_path_lengths, execution_times, distances